In [1]:
import re
import numpy as np
import pandas as pd
import seaborn as sns
from rdkit import Chem
from pathlib import Path
import matplotlib.pyplot as plt

from rdkit import RDLogger
from rdkit.Chem.rdMolDescriptors import CalcMolFormula, CalcAsphericity
from rdkit.Chem.Descriptors import ExactMolWt, NumValenceElectrons

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, KFold

from sklearn.metrics import f1_score, roc_auc_score, accuracy_score

from xgboost import XGBClassifier
from catboost import CatBoostClassifier

In [2]:
RDLogger.DisableLog('rdApp.*')
pd.set_option('display.max_columns', 20)

In [3]:
path = Path('./data')
sub_path = Path('./sub')
seed = 21

### Loading the datasets

In [4]:
# A lot of molecules...
df_train = pd.read_csv(path/'train.csv').drop(columns='ID')
df_train.Sweet = df_train.Sweet.astype(int)

df_test = pd.read_csv(path/'test.csv')

df_train.head(3)

,SMILES,Canonical SMILES,Sweet
0,OCC1OC(C(C1O)O)(CO)OC1OC(CO)C(C(C1O)O)O,OCC1OC(C(C1O)O)(CO)OC1OC(CO)C(C(C1O)O)O,1
1,ClCC1OC(C(C1O)O)(CCl)OC1OC(CO)C(C(C1O)O)Cl,ClCC1OC(C(C1O)O)(CCl)OC1OC(CO)C(C(C1O)O)Cl,1
2,COC(=O)C(NC(=O)C(CC(=O)O)N)Cc1ccccc1,COC(=O)C(NC(=O)C(CC(=O)O)N)Cc1ccccc1,1


In [5]:
# Check the nans/duplicates
df_train.isna().sum().values, df_train.duplicated().sum()

(array([0, 0, 0]), 0)

In [6]:
# Our datasets is balanced!
df_train.Sweet.value_counts()

1    1139
0    1066
Name: Sweet, dtype: int64

### Let's check what element we have in our molecules

In [7]:
# concatenate for comfort
df_mol = pd.concat([df_train.iloc[:, :2], df_test], ignore_index=True).copy(True)
df_mol.sort_index(inplace=True)

In [8]:
mols = df_mol.SMILES.apply(Chem.MolFromSmiles).values
#a = [[atom.GetSymbol() for atom in mol.GetAtoms()] for mol in mols if mols is not None]

symbols = set()
for mol in mols:
    if mol is None:
        continue
    
    for atom in mol.GetAtoms():
        symbols.add(atom.GetSymbol())

symbols = list(symbols)
symbols ### mmm, Zn....

['Zn',
 'Ag',
 'Si',
 'Hg',
 'Mg',
 'O',
 'Ar',
 'Pb',
 'Ti',
 'H',
 'P',
 'Ne',
 'K',
 'As',
 'Ba',
 'He',
 'Bi',
 'Sr',
 'Br',
 'N',
 'Cl',
 'C',
 'Ca',
 'S',
 'Fe',
 'Na',
 'Rn',
 'F',
 'I',
 'Li']

In [9]:
re.findall(r'[A-Z][a-z]*|\d+', re.sub('[A-Z][a-z]*(?![\da-z])', r'\g<0>1', 'C12H22O11'))

['C', '12', 'H', '22', 'O', '11']

### Enhancing the dataset with new features

#### Add "count of atoms", "count of valence electrons", "count of aromatic atoms" features

In [10]:
ext_f = []

In [11]:
def get_atoms_count(formula: str):
    global symbols, sym_re
    
    stats = {el: 0 for el in symbols}
    sym_data = re.findall(r'[A-Z][a-z]*|\d+', re.sub('[A-Z][a-z]*(?![\da-z])', r'\g<0>1', formula))
    sym_data = [s for s in sym_data if s != '']
    
    l = len(sym_data)
    
    for i in range(0, l, 2):
        if i + 1 >= l:
            sym_data.append('1')
            
        stats[sym_data[i]] = int(sym_data[i+1])
    df = pd.DataFrame(stats, index=[0])
    df.columns = df.columns + '_count'
    return df.iloc[0, :]

def extend_dataset_with_atom_counts(df):
    df = df.copy(True)
    
    # Create formulae
    mols = df.SMILES.apply(Chem.MolFromSmiles).dropna()
    
    formulas = mols.map(CalcMolFormula) 
    valence_electrons = mols.map(NumValenceElectrons)
    aromatic = mols.map(lambda x: x.GetAromaticAtoms()).map(len)
    
    df['formula'] = formulas
    df['valence_electrons'] = valence_electrons
    df['aromatic_atoms_count'] = aromatic
    
    df['formula'].fillna('', inplace=True)
    df['valence_electrons'].fillna(df.valence_electrons, inplace=True)
    
    # Add element counts
    df_train_ext = df.formula.apply(get_atoms_count) #.iloc[:, :-1]
    
    return pd.concat([df, df_train_ext], axis=1)

ext_f.append(extend_dataset_with_atom_counts)

#### Add weight feature

In [12]:
def extend_dataset_with_mol_weights(df):
    df = df.copy(True)
    
    weights = df.SMILES.apply(Chem.MolFromSmiles).dropna().map(ExactMolWt)
    df['weight'] = weights
    df.weight.fillna(df.weight.mean(), inplace=True)
    
    return df

ext_f.append(extend_dataset_with_mol_weights)

def ext_pipeline(df, ext_f):
    for f in ext_f:
        df = f(df)
    return df

In [13]:
df_train = ext_pipeline(df_train, ext_f)
df_test = ext_pipeline(df_test, ext_f)
df_train.head(3)

,SMILES,Canonical SMILES,Sweet,formula,valence_electrons,aromatic_atoms_count,Zn_count,Ag_count,Si_count,Hg_count,...,Ca_count,S_count,Fe_count,Na_count,Rn_count,F_count,I_count,Li_count,2_count,weight
0,OCC1OC(C(C1O)O)(CO)OC1OC(CO)C(C(C1O)O)O,OCC1OC(C(C1O)O)(CO)OC1OC(CO)C(C(C1O)O)O,1,C12H22O11,136.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,342.116212
1,ClCC1OC(C(C1O)O)(CCl)OC1OC(CO)C(C(C1O)O)Cl,ClCC1OC(C(C1O)O)(CCl)OC1OC(CO)C(C(C1O)O)Cl,1,C12H19Cl3O8,136.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,396.014551
2,COC(=O)C(NC(=O)C(CC(=O)O)N)Cc1ccccc1,COC(=O)C(NC(=O)C(CC(=O)O)N)Cc1ccccc1,1,C14H18N2O5,114.0,6.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,294.121572


### Create the pipeline

In [14]:
target_col = 'Sweet'

num_cols = [
    'valence_electrons', 'aromatic_atoms_count', 'N_count',
    'S_count', 'H_count', 'Rn_count', 'As_count', 'K_count',
    'Ba_count', 'Bi_count', 'Br_count', 'Ne_count', 'Na_count',
    'C_count', 'O_count', 'I_count', 'Ti_count', 'Pb_count',
    'Li_count', 'F_count', 'Fe_count', 'P_count', 'Sr_count',
    'Ca_count', 'He_count', 'Hg_count', 'Zn_count', 'Cl_count',
    'Si_count', 'Ag_count', 'Mg_count', 'Ar_count', 'weight'
]

In [15]:
X_train, X_test, y_train, y_test = train_test_split(
    df_train[num_cols], df_train.Sweet,
    test_size=0.05, random_state=seed
)
X_train.head(2)

,valence_electrons,aromatic_atoms_count,N_count,S_count,H_count,Rn_count,As_count,K_count,Ba_count,Bi_count,...,Ca_count,He_count,Hg_count,Zn_count,Cl_count,Si_count,Ag_count,Mg_count,Ar_count,weight
1924,60.0,0.0,4.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,158.043990
42,86.0,6.0,2.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,226.058971


In [16]:
clf_model = CatBoostClassifier(silent=True)

grid_params = {
    'clf__depth':    range(2, 10, 1),
    'clf__iterations': [10,20,30,40,50,60,70,80,90,100],
    'clf__learning_rate': [0.1, 0.01, 0.05, 0.03],
    'clf__l2_leaf_reg': [1, 3, 5, 7, 9]
}

cv = KFold(
    5,
    random_state=seed,
    shuffle=True
)

num_trf = Pipeline([
    ('imp', SimpleImputer(strategy='mean')), # mb I'l add more columns
    ('scaler', StandardScaler())
])

clf_pipeline = Pipeline([
    ('trf', num_trf),
    ('clf', clf_model),
])


clf = GridSearchCV(
    estimator=clf_pipeline,
    param_grid=grid_params,
    
    scoring='f1',
    
    n_jobs=-1,
    cv=cv,
    
    verbose=True,
    refit=True,
)

In [17]:
def show_metrics(clf, X, y_test):
    y_predict = np.argmax(clf.predict_proba(X), axis=1)
    y_proba = clf.predict_proba(X)[:, 1]
    
    m_f1 = f1_score(y_test, y_predict)
    m_rau = roc_auc_score(y_test, y_proba)
    m_acc = accuracy_score(y_test, y_predict)
    
    out = {
        'f1': m_f1,
        'roc_auc': m_rau,
        'accuracy': m_acc,
    }
    
    print(out)

#### Fit the model, get the metrics. Repeat the process (and get best estimator)

In [18]:
clf.fit(X_train, y_train)
show_metrics(clf, X_test, y_test)

Fitting 5 folds for each of 1600 candidates, totalling 8000 fits
{'f1': 0.8181818181818182, 'roc_auc': 0.9029220779220779, 'accuracy': 0.8198198198198198}


In [19]:
df_out = df_test.copy(True)
df_out['Sweet'] = clf.predict(df_test[num_cols]).astype(bool)

In [21]:
df_out[['ID', 'Sweet']].to_csv(sub_path/'catboost_small_features_f1.csv', index=False)